In [7]:
from datetime import datetime, timedelta, timezone
import googlemaps
from dotenv import load_dotenv
import os
import requests
import cudf
import pandas as pd

# Initialize the Google Maps client with your API key
load_dotenv()
api_key=os.environ.get('GOOGLE_MAPS_API_KEY')
gmaps = googlemaps.Client(key=api_key)

In [8]:
start_address_astoria='2167 19th st, Astoria, NY 11105'
start_address_brooklyn='784 President St, Brooklyn, NY 11215'

# Load the CSV file
file_path = 'japanese_restaurants.csv'
restaurants_df = pd.read_csv(file_path)

# Create a new DataFrame to store cleaned data
cleaned_restaurants_df = pd.DataFrame(columns=restaurants_df.columns)

# Process each row
for index, row in restaurants_df.iterrows():
    restaurant_name = row['name']
    restaurant_address = row['address']

    if "Multiple Locations" in restaurant_address:
        # Handle multiple locations by finding actual addresses
        try:
            places_result = gmaps.places(query=restaurant_name, type='restaurant')
            for place in places_result['results']:
                new_address = place['formatted_address']
                new_row = row.copy()
                new_row['address'] = new_address
                cleaned_restaurants_df = pd.concat([cleaned_restaurants_df, pd.DataFrame([new_row])], ignore_index=True)
        except Exception as e:
            print(f"Error processing multiple locations for {restaurant_name}: {e}")
    else:
        # Verify the address
        try:
            geocode_result = gmaps.geocode(restaurant_address)
            if geocode_result:
                cleaned_restaurants_df = pd.concat([cleaned_restaurants_df, pd.DataFrame([row])], ignore_index=True)
            else:
                print(f"Invalid address, skipping: {restaurant_address}")
        except Exception as e:
            print(f"Error verifying address for {restaurant_name}: {e}")

# Save the cleaned data to a new CSV file
cleaned_file_path = 'cleaned_japanese_restaurants.csv'
cleaned_restaurants_df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned data saved to {cleaned_file_path}")

# List to hold restaurants meeting both travel time and rating constraints
qualified_restaurants = []

for index, row in cleaned_restaurants_df.iterrows():
    restaurant_name = row['name']
    restaurant_address = row['address']

    try:
        # Geocode the restaurant address to get latitude and longitude
        geocode_result = gmaps.geocode(restaurant_address)

        if not geocode_result or len(geocode_result) == 0:
            print(f"Could not geocode address: {restaurant_address}")
            continue

        lat = geocode_result[0]['geometry']['location']['lat']
        lng = geocode_result[0]['geometry']['location']['lng']
        destination = f"{lat},{lng}"

        # Get details of the place using Place Search API
        place_results = gmaps.places(query=f"{restaurant_name} {restaurant_address}")

        if not place_results['results']:
            print(f"No places found for: {restaurant_name} at {restaurant_address}")
            continue

        # Extract the place_id of the first result
        place_id = place_results['results'][0]['place_id']

        # Get details of the place using Place Details API
        place_details = gmaps.place(place_id=place_id, fields=['rating'])

        if 'rating' not in place_details['result']:
            print(f"No rating found for: {restaurant_name} at {restaurant_address}")
            continue

        # Extract the rating from the response
        restaurant_rating = place_details['result']['rating']

        # Skip restaurants with a rating less than 4.5
        if restaurant_rating < 4.5:
            continue

        # Calculate travel time from Brooklyn
        directions_brooklyn = gmaps.directions(
            start_address_brooklyn,
            destination,
            mode="transit",
            transit_mode="subway",
            arrival_time=arrival_time
        )

        if not directions_brooklyn or len(directions_brooklyn[0]['legs']) == 0:
            print(f"No directions found for {restaurant_name} from Brooklyn")
            continue

        time_brooklyn = round(directions_brooklyn[0]['legs'][0]['duration']['value'] / 60)

        # Calculate travel time from Astoria
        directions_astoria = gmaps.directions(
            start_address_astoria,
            destination,
            mode="transit",
            transit_mode="subway",
            arrival_time=arrival_time
        )

        if not directions_astoria or len(directions_astoria[0]['legs']) == 0:
            print(f"No directions found for {restaurant_name} from Astoria")
            continue

        time_astoria = round(directions_astoria[0]['legs'][0]['duration']['value'] / 60)

        # Check if both travel times meet the constraints
        if time_brooklyn <= 45 and time_astoria <= 45:
            qualified_restaurants.append({
                'name': restaurant_name,
                'rating': restaurant_rating,
                'time_brooklyn': time_brooklyn,
                'time_astoria': time_astoria
            })

    except Exception as e:
        print(f"Error fetching details for {restaurant_name}: {e}")

print('\u2501' * 100)
print("Qualified Restaurants")
print('\u2501' * 100)

for restaurant in qualified_restaurants:
    print(f"Restaurant: {restaurant['name']}, Rating: {restaurant['rating']}")
    print(f"Travel time - Brooklyn: {restaurant['time_brooklyn']} mins, Astoria: {restaurant['time_astoria']} mins")


Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Invalid address, skipping: Multiple locations
Cleaned data saved to cleaned_japanese_restaurants.csv
No rating found for: Akiko at 232 E 9th St, New York, NY 10003
No places found for: Gaijin at 870 9th Ave, New York, NY 10019
No places found for: Omen Azen at 152 9th Ave, New York, NY 10011
No places found for: Udon West at 248 W 52nd St, New York, NY 10019
No rating found for: Robataya NY at 231 E